### Piston model:

complex pressure *$Pt(pz)$* at a point *$pz$* due to a flat circular piston emitter at point *$pt$*

### $P_t(p_z) = \frac{P_{ref}}{d(p_z, p_t)} \cdot \frac{2 \cdot J_1(k\cdot r\cdot \textrm{sin} \theta_{zt})}{k\cdot r\cdot \textrm{sin} \theta_{zt}}\cdot e^{i(\phi_t + k\cdot d(p_z,p_t))}$

$ P_{ref} $ is a constant that is defined by emitter amplitude, 

### $P_{ref} = A \cdot V_{pp} $

where $ A $ is the emitter output efficiency

and $ V_{pp} $ is the excitation signal peak-to-peak amplitude 

$ d(p_z, p_t) $ is the Euclidean distance between point $ p_z $ and the center of the emitter, $ p_t $ 

The directivity function for a emitter depends on the angle $\theta$ between the emitter normal and point $ p_z $ 

### $ D_f = \frac{2 \cdot J_1(k\cdot r\cdot \textrm{sin} \theta_{zt})} {k\cdot r\cdot \textrm{sin} \theta_{zt}} $

where $ J_1 $ is the Bessel function of the first kind, 

$ k=2\pi/\lambda $ is the wavenumber, 

$ r $ is the emitter radius, 

$ \theta_{zt} $ is the polar angle between points $p_z$ and $p_t$, 

$ \phi_t $ is the initial phase of the emitter



In [49]:
import sys
import math as math
import numpy as np

In [60]:

rho_a = 1.184                           # density of air at 25°C (kg/m3) 
c = 346.13                              # speed of sound in air at 25°C (m/s)
f = 40000                               # frequency (Hz)

_lambda = lambda f: c/f                 # wavelength (m)

_k = lambda f: 2 * math.pi/_lambda(f)     # wavenumber 

# Euclidean distance, 2-norm or magnitude of the vector, sqrt of the inner product of a vector with itself 
_distance = lambda pz, pt: math.sqrt( (pz[0]-pt[0])**2 + (pz[1]-pt[1])**2 + (pz[2]-pt[2])**2 )  

# cross product of pz and pt over the distance between pz
_sin_theta = lambda pz, pt: math.sqrt( (pz[0]-pt[0])**2 + (pz[1]-pt[1])**2 ) / _distance(pz, pt)  


def hexagon_diameter_to_coordinates(d, x_spacing=10.5/1000, y_spacing=9/1000) -> list((float, float, float)): 
    """
    Coordinate system for d-transducers diameter hexagon. Centrepoint of central transducer is at origin (0,0,0).
    Array begins with the bottom left transducer.
    
    args: 
        d:          diameter of hexagon (longest row) in transducer units 
        x_spacing:  interspacing between elements in the x axis
        y_spacing:  interspacing between elements in the y axis
        f_tran:     focal length of the PAT [m]
    
    returns:
        coords: nx3 array of coords for this hexagon, with [0, 0, 0] as the centrepoint.
    """
    
    # from the diameter in transducer units (central and longest row) calculate array with transducers count 
    # for bottom row up to central row:    
    bottom_to_central_row_tran_count = np.arange(np.floor((d+1)/2), np.floor(d+1), 1, dtype=int)

    # calculate array with rows' transducers count:
    rows_transducer_count = np.concatenate( (   bottom_to_central_row_tran_count,
                                                np.flip( bottom_to_central_row_tran_count )[1:]), 
                                            axis=0)
    coords = []   
    # for each row, depending on whether it is offset or not (i.e. shifted in relation to central row), 
    # calculate and assign X Y coordinates to each transducer:
    for row, row_length in enumerate(rows_transducer_count):
        for elem in range(row_length):      
            coord_x = x_spacing * ( elem - row_length/2 + .5 )
            coord_y = -sys.maxsize - 1
            coord_z = 0
            
            if d % 2 != 0:
                coord_y = y_spacing * (row - (d-1)/2)
            else:
                coord_y = y_spacing * (row - d/2)
                
            coords.append((coord_x, coord_y, coord_z))  
    
    return np.array(coords)


def directivity(sin_theta, k, r=4.5/1000):

    # argument of 1st order Bessel function
    bessel_J1 = k*r*sin_theta
    
    # taylor expansion of first order Bessel function over its agrument — J_1(bessel_J1)/bessel_J1
    # wolframalpha.com – Series[BesselJ[1,x]/x,{x,0,10}] 
    taylor_exp = (1/2)-(bessel_J1**2/16)+(bessel_J1**4/384)-(bessel_J1**6/18432)+(bessel_J1**8/1474560)-(bessel_J1**10/176947200)+(bessel_J1**12/29727129600) 
    
    return 2 * taylor_exp


def reference_pressure(A=0.17, V=20): 
    """
    A — emitter output efficiency (Pa/m*V) 
    V — excitation signal peak-to-peak amplitude (Vpp)  
    """
    return A * V    

def pressure(p_ref, distance, sin_theta, k, r ):
    return p_ref * 1/distance * directivity(sin_theta, k, r) * np.exp(1j*k*distance)

# reference sound pressure level for airborne sound is 20 micropascals (μPa or e-6) or 15.849 μPa
db_spl_to_pascal = lambda db: 10**((db-20)/20e-6)

pascal_to_db_spl = lambda pa: 20*math.log10(pa/20e-6)



In [69]:

_lambda(40000)
_k(40000)
r = 4.5/1000
# pz = (0,0,10) 
# pt = (0,0,0)
# print(_distance(pz, pt))
# # print(_sin_theta(pz, pt))

# P = pressure(reference_pressure(), _distance(pz, pt), _sin_theta(pz, pt), _k(40000), r)
# abs(P)

focal_point = (0, 0, 0.1) # meters
hex_points = hexagon_diameter_to_coordinates(d=3) # list of spatial coordinates in meters 
# print(hex_points)

pressures = [ abs( pressure( reference_pressure(), _distance(focal_point, h), _sin_theta(focal_point, h), _k(40000), r) )  for h in hex_points ]  
# print(*pressures, sep='\n')
print(sum(pressures))

pascal_to_db_spl(sum(pressures))

233.98701583137074


141.36323525977448

In [ ]:
# hex(1) "34.0 Pa — 124.60897842756549 dB SPL"
# hex(2) "134.66030890167056 Pa — 136.56419221422703 dB SPL"
# hex(3) "233.98701583137074 Pa — 141.36323525977448 dB SPL"
# hex(7) "1144.6094034148803 Pa — 155.15254627644873 dB SPL"
# hex(8) "1550.8818934380347 Pa — 157.79097459900558 dB SPL"
# hex(9) "1783.4739949904563 Pa — 159.004735711321 dB SPL"